In [1]:
import timeit
start=timeit.default_timer()

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

from collections import Counter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import spacy
nlp = spacy.load('en')
from spacy.lang.en.stop_words import STOP_WORDS

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

In [6]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

## Load files

In [8]:
filename=['brady.txt','butler.txt','cutting.txt','dawson.txt','dyke.txt',
          'hughes.txt','mitchell.txt','richmond.txt','rinehart.txt','stretton.txt']


In [9]:
file=open(filename[0],'r')
text=file.read()
file.close
#text=nlp(text)

<function TextIOWrapper.close()>

In [10]:
text=text.lower()
text=text.replace("ï»¿","")
text=text.replace(","," ")
text=text.replace("\n"," ")
text=text.replace("*"," ")
text=text.replace(":"," ")
text=text.replace(":"," ")
text=re.sub("[\(\[].*?[\)\]]"," ",text)
text = re.sub(r'--',' ',text)
text= text.replace("http //www.gutenberg.net"," ")
text=text.replace("www.gutenberg.net"," ")
text=text.replace("14606-h.htm"," ")

text=text.replace("i."," ")
text=text.replace("ii."," ")
text=text.replace("iii."," ")
text=text.replace("iv."," ")
text=text.replace("v."," ")
text=text.replace("vi."," ")
text=text.replace("vi."," ")
text=text.replace("vii."," ")
text=text.replace("viii."," ")
text=text.replace("ix."," ")
text=text.replace("x."," ")
text=text.replace("xi."," ")


text=text.replace('\'',"")
text=text.replace('/',"")
text=text.replace('14606-h.z',"")
text=text.replace('iso-646-us',"")
text=text.replace('?'," ")
text = re.sub('\S*@\S*\s?',' ',text)
text=text.replace('"'," ")


text

'the project gutenberg ebook  and thus he came  by cyrus townsend brady  illustrated by walter b. everett   this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it  give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at        title  and thus he came  author  cyrus townsend brady  release date  january 5  2005     language  english  character set encoding          start of the project gutenberg ebook and thus he came      e-text prepared by robert cicconetti  melissa er-raqabi  and the project gutenberg online distributed proofreading team      note  project gutenberg also has an html version of this       file which includes the original illustrations.       see   or ip                or              and thus he came  a christmas fantasy  by  cyrus townsend brady  pictures by walter b. everett  g.p. putnams sons new york and london the knickerbocker press  1916       

In [11]:
text_list=text.split(".")

In [12]:
len(text_list)

936

In [13]:
file=open(filename[1],'r')
text=file.read()
file.close
#text=nlp(text)

<function TextIOWrapper.close()>

In [14]:
text=text.lower()
text=text.replace("ï»¿","")
text=text.replace(","," ")
text=text.replace("\n"," ")
text=text.replace("*"," ")
text=text.replace(":"," ")
text=text.replace(":"," ")
text=re.sub("[\(\[].*?[\)\]]"," ",text)
text = re.sub(r'--',' ',text)
text= text.replace("http //www.gutenberg.net"," ")
text=text.replace("www.gutenberg.net"," ")
text=text.replace("14606-h.htm"," ")

text=text.replace("i."," ")
text=text.replace("ii."," ")
text=text.replace("iii."," ")
text=text.replace("iv."," ")
text=text.replace("v."," ")
text=text.replace("vi."," ")
text=text.replace("vi."," ")
text=text.replace("vii."," ")
text=text.replace("viii."," ")
text=text.replace("ix."," ")
text=text.replace("x."," ")
text=text.replace("xi."," ")


text=text.replace('\'',"")
text=text.replace('/',"")
text=text.replace('14606-h.z',"")
text=text.replace('iso-646-us',"")
text=text.replace('?'," ")
text = re.sub('\S*@\S*\s?',' ',text)
text=text.replace('"'," ")


text

'the project gutenberg ebook of the thin santa claus  by ellis parker butler  this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you may copy it  give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at www.gutenberg.org   title  the thin santa claus        the chicken yard that was a christmas stocking  author  ellis parker butler  illustrator  may wilson preston  release date  march 6  2006    language  english       start of this project gutenberg ebook the thin santa claus         produced by jason isbell  emma morgan isbell  sankar viswanathan  and the online distributed proofreading team at http www.pgdp.net                                                  the thin                              santa claus                        the chicken yard that was                         a christmas stocking                                      by                           ellis parker 

In [15]:
text_list2=text.split(".")

In [16]:
len(text_list2)

638

In [17]:
new_list=text_list+text_list2

In [18]:
len(new_list)

1574

In [19]:
new_list

['the project gutenberg ebook  and thus he came  by cyrus townsend brady  illustrated by walter b',
 ' everett   this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever',
 '  you may copy it  give it away or re-use it under the terms of the project gutenberg license included with this ebook or online at        title  and thus he came  author  cyrus townsend brady  release date  january 5  2005     language  english  character set encoding          start of the project gutenberg ebook and thus he came      e-text prepared by robert cicconetti  melissa er-raqabi  and the project gutenberg online distributed proofreading team      note  project gutenberg also has an html version of this       file which includes the original illustrations',
 '       see   or ip                or              and thus he came  a christmas fantasy  by  cyrus townsend brady  pictures by walter b',
 ' everett  g',
 'p',
 ' putnams sons new york and london the knicker

In [20]:
df=pd.DataFrame(index=range(0,1574),columns=['author'])

In [21]:
df['author'][0:936]='brady'

In [22]:
df['author'][936:1574]='butler'

In [23]:
df

,author
0,brady
1,brady
2,brady
3,brady
4,brady
5,brady
6,brady
7,brady
8,brady
9,brady


In [24]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000)

In [25]:
train_data_features = vectorizer.fit_transform(new_list)
train_data_features = train_data_features.toarray()

In [26]:
train_data_features.shape

(1574, 2977)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(train_data_features, 
                                                    df['author'],
                                                    test_size=0.25,
                                                    random_state=0)

In [31]:
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(X_train,y_train)

In [32]:
forest.score(X_train,y_train)

0.8932203389830509

In [33]:
forest.score(X_test,y_test)

0.7538071065989848

In [27]:
x

NameError: name 'x' is not defined